In [1]:
import findspark
findspark.init() # this must be executed before the below import
from pyspark.sql import SparkSession
import pyarrow as pa
import pyarrow.parquet as pq
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Python Spark SQL Execution") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory","8g") \
    .config("spark.memory.offHeap.enabled",True) \
    .config("spark.memory.offHeap.size","8g") \
    .getOrCreate()
fs = pa.hdfs.connect(host='10.77.110.133', port=9001, user='liupengju')
import numpy as np
import time
from partition_tree import PartitionTree
from partition_algorithm import PartitionAlgorithm
from join_until import JOIN_UNTIL
import pickle

/home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning
/home/liupengju/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from ipykernel import kernelapp as app


In [2]:
def query_with_parquets(query,hdfs_path, querytype=0, partition_tree=None,pids=[],print_execution_time=False):
    start_time = time.time()
    sql=""
    where_clause=''
    if query is not None:
        for i, dim in enumerate(used_dims):
            where_clause += column_name_dict[dim] + '>' + str(query[i]) + ' and '
            where_clause += column_name_dict[dim] + '<' + str(query[len(used_dims) + i]) + ' and '
        where_clause = where_clause[0:-4]  # remove the last 'and '
    pids_str = str(set(pids)).replace(" ", "")  # '{1,2,3}'
    pq_hdfs_path = hdfs_path + '/partition_' + pids_str + ".parquet"
    if querytype == 0:
        sql=[]
        for pid in pids:
#             "SELECT * FROM parquet.`" + pq_hdfs_path + "`WHERE " + where_clause
            sql.append(f"SELECT * FROM parquet.`{hdfs_path + '/partition_' + str(pid) + '.parquet'}`"+ "WHERE " + where_clause)
    elif querytype == 1:
        sql = "SELECT COUNT(*) FROM parquet.`" + pq_hdfs_path + "`WHERE " + where_clause
    elif querytype == 2:
        sql = "SELECT _c0 FROM parquet.`" +pq_hdfs_path + "`"
    elif querytype==3:
        sql=[]
        for pid in pids:
            sql.append(f"SELECT _c0 FROM parquet.`{hdfs_path + '/partition_' + str(pid) + '.parquet'}`")

    # print("generated sql:", sql)
    end_time_1 = time.time()
#     query_result = spark.sql(sql).collect()
    if isinstance (sql,list):
        for part_sql in sql:
            spark.sql(part_sql).collect()
    else:
        query_result=spark.sql(sql).collect()
    end_time_2 = time.time()

    parquets_path=[hdfs_path + '/partition_' + str(pid) + ".parquet" for pid in pids]
    actual_data_size=0
    data_size_list=[]
    for par_path in parquets_path:
        fw=fs.open(par_path,'rb')
        meta = pa.parquet.read_metadata(fw, memory_map=False).to_dict()
        actual_data_size+=meta['num_rows']
        data_size_list.append(meta['num_rows'])
        fw.close()
    query_translation_time = end_time_1 - start_time
    query_execution_time = end_time_2 - end_time_1
    print('query execution time: ', query_execution_time)
    if not isinstance(sql,list):
        print(pids)
        print(f"{data_size_list}--{len(query_result)}")
    else:
        print(f"{data_size_list} -> {sum(data_size_list)}")
    if print_execution_time:
        print('query translation time: ', query_translation_time)
        print('query execution time: ', query_execution_time)

    # return (query_result, query_translation_time, query_execution_time) # this takes too much memory
    return (query_translation_time, query_execution_time,0,actual_data_size)

In [3]:
# path="hdfs://10.77.110.133:9001/par_nora/join/AdaptDB/prob2/scale1/merged_A/merged/partition_{115}.parquet"
# result1=spark.sql(f"SELECT _c0 FROM parquet.`{path}`"）
# result2=spark.sql(f"SELECT _c0 FROM parquet.`{path}`"）

# print(len(result))


In [4]:
def batch_query(a_training_set,b_training_set,a_join_queries, b_join_queries,A_partition_path,B_partition_path,group_type,hdfs_path_a,hdfs_path_b):
    
    pa_A = PartitionAlgorithm()
    pa_A.partition_tree = PartitionTree(len(used_dims))
    pa_B = PartitionAlgorithm()
    pa_B.partition_tree = PartitionTree(len(used_dims))
    
    pa_A.partition_tree.join_attr,pa_B.partition_tree.join_attr=0,0
    pa_A.partition_tree.load_tree(A_partition_path)
    pa_B.partition_tree.load_tree(B_partition_path)
    total_query_exection_time1,totol_real_data_size1=0,0
    # record time for normal queries
    for i,query_set in enumerate([a_training_set,b_training_set]):
        tree=pa_A.partition_tree if i==0 else pa_B.partition_tree
        cur_hdfs_path=hdfs_path_a if i==0 else hdfs_path_b
        for count,query in enumerate(query_set):
            pids=tree.query_single(query)
            print('process query#', count)
            result=query_with_parquets(query,hdfs_path=cur_hdfs_path,querytype=0,partition_tree=tree,pids=pids)
            total_query_exection_time1+=result[1]
            totol_real_data_size1+=result[3]
    # record time for join queries
    total_query_exection_time2,totol_real_data_size2=0,0
    ju.set_partitioner(pa_A, pa_B)
    time0=time.time()
    a_ids_for_q,b_ids_for_q= ju.print_shuffle_hyper_blocks(a_join_queries, b_join_queries,group_type)
    total_query_exection_time2+=time.time()-time0
    for q_no,total_a_ids in enumerate(a_ids_for_q):
        print('process query#', q_no)
        total_b_ids=b_ids_for_q[q_no]
        print(f"total_b -> {total_b_ids}")
        my_results=[]
        result_a=query_with_parquets(None,hdfs_path=hdfs_path_a,querytype=2,partition_tree=pa_A.partition_tree,pids=total_a_ids)
        my_results.append(result_a)
        for b_ids in total_b_ids:
            result_b=query_with_parquets(None,hdfs_path=hdfs_path_b,querytype=2,partition_tree=pa_B.partition_tree,pids=b_ids)
            my_results.append(result_b)
        for result in my_results:
            total_query_exection_time2+=result[1]
            totol_real_data_size2+=result[3]
    total_query_exection_time=[total_query_exection_time1,total_query_exection_time2]
    totol_real_data_size=[totol_real_data_size1,totol_real_data_size2]
    query_amount=len(a_training_set)+len(b_training_set)+len(a_join_queries)
    
    print("Query execution time:",sum(total_query_exection_time)/query_amount,total_query_exection_time)
    print("Access row count:",sum(totol_real_data_size),totol_real_data_size)

In [5]:
# ==== set environment parameters and generate dataset ====
scale_factor = 1
problem_type = 2
used_dims = [1,2,4] #tpch
# used_dims = [1,2,3] #tpcds

num_dims = 16
join_attr=0
column_names = ['_c'+str(i) for i in range(num_dims)]
column_name_dict = {}
for i in range(num_dims):
    column_name_dict[i] = column_names[i]
    
# query_path = '/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/NORA_experiments/queryset/join/' 
# a_training_set=np.genfromtxt(f"{query_path}a_prob{problem_type}_{scale_factor}_train.csv",delimiter=',')
# b_training_set=np.genfromtxt(f"{query_path}b_prob{problem_type}_{scale_factor}_train.csv",delimiter=',')
# with open(f"{query_path}join_a_prob{problem_type}_{scale_factor}_train",'rb') as f:
#     a_join_queries=pickle.load(f)
# with open(f"{query_path}join_b_prob{problem_type}_{scale_factor}_train",'rb') as f:
#     b_join_queries=pickle.load(f)    

# scale 100/ 50 and 10
# hdfs_base_path = 'hdfs://10.77.110.133:9001/par_nora/join/'
# hdfs_path_nora_a = hdfs_base_path + 'NORA/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/merged_A/merged"
# hdfs_path_nora_b = hdfs_base_path + 'NORA/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/merged_B/merged"
# hdfs_path_adaptdb_a = hdfs_base_path + 'AdaptDB/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/merged_A/merged"
# hdfs_path_adaptdb_b = hdfs_base_path + 'AdaptDB/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/merged_B/merged"

# newly added
querytype = 2 # 0: SELECT *;  2: SELECT variance(_c0)
# partition_base_path = '/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/PartitionLayout/join/'
# scale 100
# jnora_A_partition_path = partition_base_path + 'prob' + str(problem_type) + '_jnora_A_scale' + str(scale_factor)
# jnora_B_partition_path = partition_base_path + 'prob' + str(problem_type) + '_jnora_B_scale' + str(scale_factor)
# adaptdb_A_partition_path = partition_base_path + 'prob' + str(problem_type) + '_adaptdb_A_scale' + str(scale_factor)
# adaptdb_B_partition_path = partition_base_path + 'prob' + str(problem_type) + '_adaptdb_B_scale' + str(scale_factor)

ju = JOIN_UNTIL(None, None, join_attr, len(used_dims))

In [6]:
# batch_query(a_training_set,b_training_set,a_join_queries, b_join_queries,adaptdb_A_partition_path,adaptdb_B_partition_path,1,hdfs_path_adaptdb_a,hdfs_path_adaptdb_b)

In [7]:
# batch_query(a_training_set,b_training_set,a_join_queries, b_join_queries,jnora_A_partition_path,jnora_B_partition_path,3,hdfs_path_nora_a,hdfs_path_nora_b)


In [8]:
# execute queries for multi-tables
table_num=5
base_join_depth=2
# # tpch
base_join_path='/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/NORA_experiments/queryset/multi_join/'
partition_base_path = '/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/PartitionLayout/multi_join/'
multi_hdfs_base_path='hdfs://10.77.110.133:9001/par_nora/multi_join/'
# tpcds
# base_join_path='/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/NORA_experiments/queryset/multi_join/tpcds/'
# partition_base_path = '/home/liupengju/pycharmProjects/NORA_JOIN_SIMULATION/PartitionLayout/multi_join/tpcds/'
# multi_hdfs_base_path='hdfs://10.77.110.133:9001/par_nora/multi_join/tpcds/'
algo_names=['adaptdb','jnora']
hdfs_algo_names=['AdaptDB','NORA']

In [9]:
def batch_join_query(tab_num,algo_name,hdfs_algo_name,group_type):
    pa_list=[]
    total_query_exection_time,totol_real_data_size=[],[]
    total_query_amount=0
    # record time for single-table queries
    for tid in range(tab_num):
        training_set = np.genfromtxt(f"{base_join_path}{tid}_prob2_{scale_factor}_train.csv", delimiter=',')
        total_query_amount+=len(training_set)
        pa = PartitionAlgorithm()
        pa.partition_tree = PartitionTree(len(used_dims))
        pa.partition_tree.join_attr = 0
        pa.partition_tree.load_tree(f'{partition_base_path}prob2_{algo_name}_{tid}_scale{scale_factor}')
        pa_list.append(pa)
        cur_hdfs_path=f'{multi_hdfs_base_path}{hdfs_algo_name}/prob{problem_type}/scale{scale_factor}/{tid}/merged'
        temp_query_exection_time,temp_real_data_size=0,0
        for qno,query in enumerate(training_set):
            print('process query#', qno)
            pids=pa.partition_tree.query_single(query)
            result=query_with_parquets(query,hdfs_path=cur_hdfs_path,querytype=0,partition_tree=pa.partition_tree,pids=pids)
            temp_query_exection_time+=result[1]
            temp_real_data_size+=result[3]
        total_query_exection_time.append(temp_query_exection_time)
        totol_real_data_size.append(temp_real_data_size)

    # record time for join queries
    total_group_time=0
    for link in range(tab_num-1):
        target=link+1
        pa_A=pa_list[link]
        pa_B=pa_list[target]
        with open(f"{base_join_path}join_{link}_{target}_a_prob2_{scale_factor}_train", 'rb') as f:
            a_join_queries = pickle.load(f)
        with open(f"{base_join_path}join_{link}_{target}_b_prob2_{scale_factor}_train", 'rb') as f:
            b_join_queries = pickle.load(f)
        ju.set_partitioner(pa_A, pa_B)
        time0=time.time()
        a_ids_for_q,b_ids_for_q= ju.print_shuffle_hyper_blocks(a_join_queries, b_join_queries,group_type)
        total_group_time+=time.time()-time0
        total_query_amount+=len(a_join_queries)
        temp_query_exection_time,temp_real_data_size=0,0
        for q_no,total_a_ids in enumerate(a_ids_for_q):
            print('process query#', q_no)
            total_b_ids=b_ids_for_q[q_no]
            print(f"total_a -> {total_a_ids}")
            print(f"total_b -> {total_b_ids}")
            my_results=[]
            result_a=query_with_parquets(None,hdfs_path=f'{multi_hdfs_base_path}{hdfs_algo_name}/prob{problem_type}/scale{scale_factor}/{link}/merged',querytype=2,partition_tree=pa_A.partition_tree,pids=total_a_ids)
            my_results.append(result_a)
            for b_ids in total_b_ids:
                result_b=query_with_parquets(None,hdfs_path=f'{multi_hdfs_base_path}{hdfs_algo_name}/prob{problem_type}/scale{scale_factor}/{target}/merged',querytype=2,partition_tree=pa_B.partition_tree,pids=b_ids)
                my_results.append(result_b)
            for result in my_results:
                temp_query_exection_time+=result[1]
                temp_real_data_size+=result[3]
        total_query_exection_time.append(temp_query_exection_time)
        totol_real_data_size.append(temp_real_data_size)
    avg_exection_time=(sum(total_query_exection_time)+total_group_time)/total_query_amount
    print("Query execution time:",avg_exection_time,total_query_exection_time)
    print("Access row count:",sum(totol_real_data_size),totol_real_data_size)
    return avg_exection_time

In [10]:
#Vary the number of joining tables and execute the queries in turn
final_latency_res={}
for tab_num in range(2,table_num+1):
# for tab_num in [2,5]:
    adapt_time=batch_join_query(tab_num,algo_names[0],hdfs_algo_names[0],1)
    nora_time=batch_join_query(tab_num,algo_names[1],hdfs_algo_names[1],3)
    final_latency_res[tab_num]=[adapt_time,nora_time]
print(final_latency_res)

process query# 0
query execution time:  5.592221975326538
[15454] -> 15454
process query# 1
query execution time:  0.8723926544189453
[10913, 15609] -> 26522
process query# 2
query execution time:  0.3588705062866211
[23846] -> 23846
process query# 3
query execution time:  0.9502792358398438
[20990, 21457, 22762] -> 65209
process query# 4
query execution time:  0.9632327556610107
[18249, 12840, 10154] -> 41243
process query# 5
query execution time:  1.8501098155975342
[21765, 16230, 18463, 20021, 10224, 18932, 10779] -> 116414
process query# 6
query execution time:  0.6242856979370117
[11345, 10802] -> 22147
process query# 7
query execution time:  1.4906363487243652
[17216, 10206, 21728, 10821] -> 59971
process query# 8
query execution time:  0.25538134574890137
[29312] -> 29312
process query# 9
query execution time:  0.9163186550140381
[11984, 18966, 13284, 10301] -> 54535
process query# 10
query execution time:  1.2298965454101562
[14179, 21218, 11725, 15022, 13728] -> 75872
process 

query execution time:  0.3447685241699219
[16771, 10566] -> 27337
process query# 95
query execution time:  0.3934633731842041
[10929, 11063] -> 21992
process query# 96
query execution time:  0.619964599609375
[10845, 13176, 18623, 24380] -> 67024
process query# 97
query execution time:  0.3155784606933594
[10746, 13728] -> 24474
process query# 98
query execution time:  0.6581141948699951
[16209, 11812, 12598] -> 40619
process query# 99
query execution time:  0.34140682220458984
[14026, 14180] -> 28206
process query# 100
query execution time:  0.46248841285705566
[16982, 10845] -> 27827
process query# 101
query execution time:  0.3427591323852539
[19959, 14740] -> 34699
process query# 102
query execution time:  0.36433863639831543
[17420, 19362] -> 36782
process query# 103
query execution time:  0.5012235641479492
[21718, 12577, 11632] -> 45927
process query# 104
query execution time:  0.23381590843200684
[18966] -> 18966
process query# 105
query execution time:  0.5053787231445312
[131

query execution time:  0.3049132823944092
[24380] -> 24380
process query# 186
query execution time:  0.3080253601074219
[12191, 15543] -> 27734
process query# 187
query execution time:  0.32212042808532715
[24648, 16982] -> 41630
process query# 188
query execution time:  0.13902521133422852
[13161] -> 13161
process query# 189
query execution time:  0.13984394073486328
[15776] -> 15776
process query# 190
query execution time:  0.9354643821716309
[18972, 13292, 10469, 10371, 13079, 15899] -> 82082
process query# 191
query execution time:  0.3084127902984619
[17270, 11870] -> 29140
process query# 192
query execution time:  0.33237576484680176
[20936, 10438] -> 31374
process query# 193
query execution time:  0.29109644889831543
[16674, 19922] -> 36596
process query# 194
query execution time:  0.93882155418396
[10623, 13724, 11976, 10499] -> 46822
process query# 195
query execution time:  1.258660078048706
[10409, 14303, 14168, 21718, 11632, 16755] -> 88985
process query# 196
query executio

query execution time:  0.6905195713043213
[14191, 15452, 14631, 10320] -> 54594
process query# 71
query execution time:  0.1675710678100586
[12574] -> 12574
process query# 72
query execution time:  0.3276338577270508
[16357, 20794] -> 37151
process query# 73
query execution time:  0.3344733715057373
[18102, 11908] -> 30010
process query# 74
query execution time:  0.4968297481536865
[12302, 12646] -> 24948
process query# 75
query execution time:  0.1473219394683838
[18236] -> 18236
process query# 76
query execution time:  1.4566285610198975
[13657, 13941, 11992, 12734, 10525, 14219, 19831] -> 96899
process query# 77
query execution time:  0.3144848346710205
[18236, 14405] -> 32641
process query# 78
query execution time:  0.7738051414489746
[14717, 14265, 11869, 17677, 11831, 11233] -> 81592
process query# 79
query execution time:  0.13671636581420898
[14174] -> 14174
process query# 80
query execution time:  0.3790268898010254
[12830, 17505, 14401] -> 44736
process query# 81
query execut

query execution time:  1.0437853336334229
[576, 579, 484, 341, 342, 183, 472, 347, 575]
[11478, 13080, 14833, 21539, 14971, 64535, 14246, 17135, 11651]--183468
query execution time:  0.9149937629699707
[576, 451, 141, 144, 471, 573, 574, 575]
[11478, 13102, 73377, 14191, 10320, 11504, 14631, 11651]--160254
query execution time:  0.8404655456542969
[576, 141, 144, 471, 472, 575]
[11478, 73377, 14191, 10320, 14246, 11651]--135263
process query# 1
total_a -> [640, 671, 672, 457, 459, 460, 461, 462, 641, 642, 645, 674, 463, 471, 473, 474, 402, 403, 665, 409, 668, 432, 434, 437, 407, 663, 673, 429, 430, 436, 442, 444, 427, 452, 228, 625, 626, 629, 630, 632, 586, 468, 470, 239, 245, 242, 254, 469, 216, 606, 621, 622, 623, 628, 636, 475, 247, 624, 605, 609]
total_b -> [[592, 569, 443, 444, 540, 542], [592, 569, 443, 444, 540, 542], [540, 539, 569, 443, 444, 541, 542], [541, 539, 569, 443, 444, 540, 542], [539, 569, 444, 443, 540, 541, 542], [592, 569, 443, 444, 540, 542], [539, 569, 444, 443,

...

query execution time:  0.4908573627471924
[833, 834, 727, 729, 733, 735]
[17163, 17161, 14693, 14839, 14767, 14841]--93464
query execution time:  0.39223670959472656
[729, 735, 733, 727]
[14839, 14841, 14767, 14693]--59140
query execution time:  0.49595022201538086
[833, 727, 729, 733, 735]
[17163, 14693, 14839, 14767, 14841]--76303
query execution time:  0.21335291862487793
[729, 727]
[14839, 14693]--29532
query execution time:  0.2675914764404297
[729, 727]
[14839, 14693]--29532
query execution time:  0.2890288829803467
[833, 834]
[17163, 17161]--34324
query execution time:  0.4024477005004883
[729, 727, 733, 735]
[14839, 14693, 14767, 14841]--59140
query execution time:  0.1788806915283203
[833]
[17163]--17163
process query# 15
total_a -> [831, 266, 578, 580, 594, 608, 239, 240, 241, 600, 272, 828, 832, 381, 382, 377, 378, 395, 269, 270, 584, 586, 379, 511, 827]
total_b -> [[802, 631, 427, 373, 375, 376, 381, 382], [802, 631, 427, 373, 375, 376, 381, 382], [376, 631, 382, 375]]
quer

query execution time:  0.9336256980895996
[833, 834, 439, 750, 756, 852, 342, 727, 440, 729, 733, 735]
[17163, 17161, 17837, 11196, 12542, 11593, 14252, 14693, 13742, 14839, 14767, 14841]--174626
query execution time:  0.6821017265319824
[833, 834, 750, 756, 852, 342, 439, 440]
[17163, 17161, 11196, 12542, 11593, 14252, 17837, 13742]--115486
query execution time:  0.5028946399688721
[750, 852, 342, 727, 729, 733, 735]
[11196, 11593, 14252, 14693, 14839, 14767, 14841]--96181
query execution time:  0.41945624351501465
[729, 735, 733, 727]
[14839, 14841, 14767, 14693]--59140
query execution time:  0.5617830753326416
[750, 852, 342, 727, 729, 733, 735]
[11196, 11593, 14252, 14693, 14839, 14767, 14841]--96181
query execution time:  0.22343230247497559
[729, 727]
[14839, 14693]--29532
query execution time:  0.3119020462036133
[729, 727]
[14839, 14693]--29532
query execution time:  0.5479552745819092
[833, 834, 756, 439, 440]
[17163, 17161, 12542, 17837, 13742]--78445
process query# 21
total_

query execution time:  0.2868785858154297
[200, 586, 110]
[17216, 10266, 19030]--46512
query execution time:  0.4017786979675293
[200, 586, 110]
[17216, 10266, 19030]--46512
query execution time:  0.3016483783721924
[586, 110]
[10266, 19030]--29296
query execution time:  0.19753289222717285
[200]
[17216]--17216
process query# 1
total_a -> [419, 420, 772, 843, 719, 786, 855, 858, 228, 422, 424, 357, 426, 778, 845, 358, 788, 856, 345, 350]
total_b -> [[780, 590], [780, 590]]
query execution time:  1.7354538440704346
[419, 420, 772, 843, 719, 786, 855, 858, 228, 422, 424, 357, 426, 778, 845, 358, 788, 856, 345, 350]
[16684, 19726, 13861, 12768, 15245, 9603, 11126, 10202, 11083, 15154, 16574, 10002, 17385, 11915, 10852, 15692, 20464, 11125, 11791, 34980]--296232
query execution time:  0.28210949897766113
[780, 590]
[11972, 19438]--31410
query execution time:  0.22638773918151855
[780, 590]
[11972, 19438]--31410
process query# 2
total_a -> [577, 537, 538, 535, 536, 571]
total_b -> [[709, 71

query execution time:  0.5287771224975586
[240]
[93857]--93857
query execution time:  0.5481250286102295
[240]
[93857]--93857
query execution time:  0.5955727100372314
[240, 522]
[93857, 14626]--108483
process query# 8
total_a -> [577, 579, 580, 325, 683, 684, 685, 686, 837, 838, 749, 750, 707, 708, 719, 720, 705, 706, 713, 714, 333, 571, 535, 536, 537, 538]
total_b -> [[386, 402, 404, 408, 410], [586, 110], [410, 404, 386], [709, 711, 715, 717, 718]]
query execution time:  1.9404520988464355
[577, 579, 580, 325, 683, 684, 685, 686, 837, 838, 749, 750, 707, 708, 719, 720, 705, 706, 713, 714, 333, 571, 535, 536, 537, 538]
[13900, 13838, 12688, 10532, 10332, 10320, 10324, 10322, 14341, 14335, 14057, 11196, 19487, 19469, 15245, 14227, 19481, 19479, 15527, 13951, 12355, 13747, 19592, 19567, 19584, 19569]--387465
query execution time:  0.4624812602996826
[386, 402, 404, 408, 410]
[13397, 18076, 18079, 16042, 16064]--81658
query execution time:  0.28662896156311035
[586, 110]
[10266, 19030]-

query execution time:  1.478658676147461
[228, 424, 357, 358, 345, 350, 419, 420, 772, 843, 786, 855, 778, 845, 788, 856, 719, 858, 422, 426]
[11083, 16574, 10002, 15692, 11791, 34980, 16684, 19726, 13861, 12768, 9603, 11126, 11915, 10852, 20464, 11125, 15245, 10202, 15154, 17385]--296232
query execution time:  0.5785732269287109
[577, 578, 583, 584, 780, 590, 284, 189]
[11072, 10311, 10960, 10432, 11972, 19438, 15541, 15750]--105476
query execution time:  0.6481049060821533
[577, 578, 583, 584, 780, 590, 284, 189]
[11072, 10311, 10960, 10432, 11972, 19438, 15541, 15750]--105476
query execution time:  1.1427011489868164
[577, 578, 451, 453, 583, 584, 780, 590, 755, 277, 312, 284, 189]
[11072, 10311, 16032, 15995, 10960, 10432, 11972, 19438, 15757, 12647, 22763, 15541, 15750]--188670
process query# 15
total_a -> [577, 537, 538, 535, 536, 571]
total_b -> [[423], [709, 711, 715, 717, 718, 533, 534, 375, 376, 377, 378]]
query execution time:  0.5542383193969727
[577, 537, 538, 535, 536, 57